## La bataille de boules de neige


Un étudiant du cours de physique s'engage dans une bataille de boules de neige avec un ami. Cet ami parvient à rattraper les boules et à les renvoyer immédiatement.

L'étudiant sait qu'une boule de neige peut être envoyée à deux angles de tir différents, mais avec la même vitesse, et arriver au même point d'impact. Cependant, les temps de vol sont différents. Aussi, pour gagner la partie, l'étudiant décide de jeter deux boules, à des instants différents, l'une sur une trajectoire supérieure à l'autre. La balle supérieure créera une diversion, pendant que l'ami se préparera à l'attraper, la seconde boule arrivera et les deux balles frapperont simultanément.  Si les amis sont à une distance $L$ l'un de l'autre et qu'ils lancent les boules à une vitesse initiale $v_0$:

1- Quels sont les angles de tir?

2- Combien de temps faut-il attendre avant de jeter la deuxième boule?

3- Application numérique : $L=25$ m et $v_0=20$ m$\cdot$s$^{-1}$.




In [ ]:
from notebook import notebookapp

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

import io
import base64
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

from bokeh.plotting import figure, curdoc
from bokeh.models.widgets import Slider, CheckboxButtonGroup, PreText
from bokeh.layouts import row, column, widgetbox
from bokeh.models import ColumnDataSource, Slider, Button, TextInput, Arrow, OpenHead, NormalHead, VeeHead, Label
from bokeh.plotting import figure, show, ColumnDataSource

from bokeh.io import output_notebook, show, export_png, push_notebook


output_notebook()

In [ ]:
from IPython.display import Image
Image("boules_de_neige.png")

## Solution:

Les équations du mouvement sont:
\begin{align*}
\vec a &=
\begin{pmatrix}
0 \\ -g 
\end{pmatrix}&
\vec v &=
\begin{pmatrix}
v_0\cos\alpha \\ -gt+v_0\sin\alpha
\end{pmatrix}&
\vec r &=
\begin{pmatrix}
v_0\cos\alpha \,t\\ -\frac{1}{2}gt^2+(v_0\sin\alpha)t
\end{pmatrix}
\end{align*}
Les équations du mouvement au point $L$ donnent:
$$
\left\{
\begin{array}{l}
L=v_0\cos\alpha \,t_f\\ 0=- \frac{1}{2}gt_f^2+v_0\sin\alpha\,t_f\\
\end{array}
\right.
$$


### Définition des paramètres initiaux

In [ ]:
# Définition des paramètres initiaux
v_0 = 20              # Vitesse initiale en m/s
alpha_ini = np.pi*45/180  # 45deg en radian
L_0 = 30                  # Distance initiale en m

g = 9.81 # gravité

### Définitions des fonctions auxiliers

In [ ]:
# Fonctions pour calculer les angles de tir
alpha_1 = lambda l,v: np.arcsin((g*l)/(v**2))/2
alpha_2 = lambda l,v: (-alpha_1(l,v)*2 - np.pi)/2


# Fonctions pour calculer le temps de vol en fonction du point final et de la vitesse initiale
temps = lambda l,v, alpha: l/(np.cos(alpha)*v)

# Fonction pour calculer le temps de vol en fonction de l'angle initial et de la vitesse initiale
temps_ang = lambda v,alpha: 2*v*np.sin(alpha)/g

# Fonctions pour calcules les trajetoires
x = lambda v,t,alpha: v*np.cos(alpha)*t #vecteur temps 
y = lambda v,t,alpha: -0.5*g*t**2 + v*np.sin(alpha)*t 

## CHOICE OF PROPER ANGLE TO HIT THE DESIRED POINT

The goal here is to choose the correct angle to match the desired point (where the oponent is standing). The time will not be taken into account. Both balls will arrive to the same point but may do so in different moments but we will check that later. 

We will now assume that we already have the correct angles.

According to the problem description: 

L = 25m 

$v_0$ = 20m/s

In [ ]:
def modify_doc(doc):
    
    # Définir information à visualiser
    v_0 = 20 # Vitesse initiale [m/s]
    ang1 = 20 # Angle inferièure [º]
    ang2 = 70 # Angle superièure [º]
    L_0 = 25  # Distance initiale [m]
    g = 9.81 # gravité [m/s]

    num_points = 1000

    def initialize(ang1, ang2):
        
        angles = [ang1*np.pi/180, ang2*np.pi/180]
        
        alpha_haut = np.max(angles)
        alpha_bas = np.min(angles)

        # Calculer temps de vol pour chaque angle initial
        temps_haut = temps_ang(v_0, alpha_haut)
        temps_bas = temps_ang(v_0, alpha_bas)

        t_haut = np.linspace(0,temps_haut,num_points)
        t_bas = np.linspace(0, temps_bas, num_points)

        # Calculer trajectoire des deux boules - des qu'ils sont lancés jusqu'ils arrivent à terre
        x_haut = x(v_0, t_haut, alpha_haut)
        y_haut = y(v_0, t_haut, alpha_haut)

        x_bas = x(v_0, t_bas, alpha_bas)
        y_bas = y(v_0, t_bas, alpha_bas)

        
        
        haut = {'x': x_haut, 'y': y_haut}
        bas = {'x': x_bas, 'y': y_bas}


        return haut, bas
    
  
    # Calculer les valeurs initiales
    HAUT, BAS= initialize(ang1, ang2)

    # Créer dictionnaire pour acceder aux données
    source_HAUT = ColumnDataSource(data = HAUT)
    source_BAS = ColumnDataSource(data = BAS)

    
    
    
    # Initialisation de la figure et de ses axes
    p = figure(title="Trajectoire du projectile", plot_height=360, plot_width=720,\
            x_range=(0,L_0+10), 
               background_fill_color='#ffffff')
    p.xaxis.axis_label = 'x (m)'
    p.yaxis.axis_label = 'y (m)'
    

    
    # Première trajectoire
    r_1 = p.line('x', 'y', source=source_HAUT,color="#1118cc", line_width=4.5, alpha=0.8, legend='Première boule de neige')

    # Seconde trajectoire 
    r_2 = p.line('x', 'y', source=source_BAS, color="#8778cc", line_width=4.5, alpha=0.8, legend='Deuxième boule de neige')

    point = p.circle(30, 0, color='#FF3333', legend='Point recherchée')

    # Flèche indiquant la vitesse initiale
    arrow = Arrow(end=OpenHead(line_color="firebrick", ),
                       x_start=0, y_start=0, x_end=10, y_end=10,)
    p.add_layout(arrow)
    
    
    # Définir sliders 
    slider_A1 = Slider(start=0, end=90, step=0.1,value=20, title='Alpha_1')
    slider_A2 = Slider(start=0, end=90,step=0.1,value=70, title='Alpha_2')
    
    
    
    def refresh_source(attrname, old, new):
        # Mettre à jour les valeurs à afficher 
        ang1 = slider_A1.value
        ang2 = slider_A2.value

        HAUT, BAS = initialize(ang1, ang2)

        source_HAUT.data = HAUT
        source_BAS.data = BAS
        
    slider_A1.on_change('value', refresh_source)
    slider_A2.on_change('value', refresh_source)
    
    layout = column(
        row(p),
        row(slider_A1, slider_A2)
    )

    
    doc.add_root(layout)

In [ ]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    
    base_url = os.environ['EXTERNAL_URL']
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url



def show_document(doc):
    servers = list(notebookapp.list_running_servers())[0]
    if servers['hostname'] == 'localhost':
        show(doc) 
    else:
        show(doc, notebook_url=remote_jupyter_proxy_url)

In [ ]:
show_document(modify_doc) 

In [ ]:
a = alpha_1(25,20)
print(2*20*np.sin(a)/g)
print(a)

## CHOICE OF PROPER DELAY 

Once we have the correct angle to ensure both balls will arrive to the same point, we now want to ensure they will actually arrive at the same time. To do so, we need to wait some seconds between the launch of the first and the second ball.  

According to the problem description: 

L = 25m 

$v_0$ = 20m/s

In [ ]:
def modify_doc_delay(doc):
    
    # Définir information à visualiser
    v_0 = 20 # Vitesse initiale [m/s]
    L_0 = 25  # Distance initiale [m]
    
    ang1 = alpha_1(L_0,v_0) # Angle inferièure [rad]
    ang2 = alpha_2(L_0,v_0) # Angle inferièure [rad]
    
    g = 9.81 # gravité [m/s]

    num_points = 1000
    
    delai = 10

    def initialize(delai):
        
        angles = [ang1, ang2]
        
        alpha_haut = np.max(angles)
        alpha_bas = np.min(angles)

        # Calculer temps de vol pour chaque angle initial
        temps_haut = temps_ang(v_0, alpha_haut)
        temps_bas = temps_haut - delai

        t_haut = np.linspace(0,temps_haut, num_points)
        t_bas = np.linspace(0, temps_bas, num_points)

        # Calculer trajectoire des deux boules
        x_haut = x(v_0, t_haut, alpha_haut)
        y_haut = y(v_0, t_haut, alpha_haut)

        x_bas = x(v_0, t_bas, alpha_bas)
        y_bas = y(v_0, t_bas, alpha_bas)

        
        
        haut = {'x': x_haut, 'y': y_haut}
        bas = {'x': x_bas, 'y': y_bas}


        return haut, bas
    
  
    # Calculer les valeurs initiales
    HAUT, BAS = initialize(delai)

    # Créer dictionnaire pour acceder aux données
    source_HAUT = ColumnDataSource(data = HAUT)
    source_BAS = ColumnDataSource(data = BAS)

    
    
    
    # Initialisation de la figure et de ses axes
    p = figure(title="Trajectoire du projectile", plot_height=360, plot_width=720,\
             background_fill_color='#ffffff')
    p.xaxis.axis_label = 'x (m)'
    p.yaxis.axis_label = 'y (m)'
    

    
    # Première trajectoire
    r_1 = p.line('x', 'y', source=source_HAUT,color="#1118cc", line_width=4.5, alpha=0.8, legend='Première boule de neige')

    # Seconde trajectoire 
    r_2 = p.line('x', 'y', source=source_BAS, color="#8778cc", line_width=4.5, alpha=0.8, legend='Deuxième boule de neige')

    point = p.circle(25, 0, color='#FF3333', legend='Point recherchée')

    # Flèche indiquant la vitesse initiale
    arrow = Arrow(end=OpenHead(line_color="firebrick", ),
                       x_start=0, y_start=0, x_end=10, y_end=10,)
    p.add_layout(arrow)
    
    
    # Définir sliders 
    slider_TIME = Slider(start=0, end=7, step=0.05,value=0.5, title='Délai [s]')
    
    
    
    def refresh_source(attrname, old, new):
        # Mettre à jour les valeurs à afficher 
        delai = slider_TIME.value

        HAUT, BAS = initialize(delai)

        source_HAUT.data = HAUT
        source_BAS.data = BAS
        
    slider_TIME.on_change('value', refresh_source)
    
    layout = column(
        row(p),
        row(slider_TIME)
    )

    
    doc.add_root(layout)

In [ ]:
show_document(modify_doc_delay) 

In [ ]:
## TODO: CHECK WHAT IS GOING ON WITH THE TIME!! 

## Trajectoire en fonction de la distance et le la vitesse initiale

In [ ]:
def modify_doc(doc):
    
    # Définir information à visualiser
    v_0 = 20              # Vitesse initiale en m/s
    alpha_ini = np.pi*45/180  # 45deg en radian
    L_0 = 30                  # Distance initiale en m

    g = 9.81 # gravité

    num_points = 1000

    def initialize(l,v):
        
        # est-ce qu'on peut arriver au point desirée?
        if (g*l)/(v**2) > 1: #non
            angles = [0,0]
        else:
            angles = [alpha_1(l,v), alpha_2(l,v)] #oui
            
        alpha_haut = np.max(angles)
        alpha_bas = np.min(angles)

        # Calculer temps de vol pour chaque angle initial
        temps_haut = temps(l, v, alpha_haut)
        temps_bas = temps(l, v, alpha_bas)

        t_haut = np.linspace(0,temps_haut,num_points)
        t_bas = np.linspace(0, temps_bas, num_points)

        # Calculer trAjectoire des deux boules - des qu'ils sont lancés jusqu'ils arrivent à terre
        x_haut = x(v, t_haut, alpha_haut)
        y_haut = y(v, t_haut, alpha_haut)

        x_bas = x(v, t_bas, alpha_bas)
        y_bas = y(v, t_bas, alpha_bas)
        
        y_min_haut = np.where(y_haut <= 0)[0]
        
        if y_min_haut.size > 1:
            idx_haut = y_min_haut[1]
        else:
            idx_haut = num_points
        
        y_min_bas = np.where(y_bas <= 0)[0]
        
        if y_min_bas.size > 1:
            idx_bas = y_min_bas[1]
        else:
            idx_bas = num_points
        
        
        
        haut = {'x': x_haut[:idx_haut], 'y': y_haut[:idx_haut]}
        bas = {'x': x_bas[:idx_bas], 'y': y_bas[:idx_bas]}
        point = {'x': [l], 'y':[0]}

        return haut, bas, point
    
  
    # Calculer les valeurs initiales
    HAUT, BAS, P = initialize(L_0, v_0)

    # Créer dictionnaire pour acceder aux données
    source_HAUT = ColumnDataSource(data = HAUT)
    source_BAS = ColumnDataSource(data = BAS)
    source_P = ColumnDataSource(data = P)
    
    
    
    # Initialisation de la figure et de ses axes
    p = figure(title="Trajectoire du projectile", plot_height=360, plot_width=720,\
            x_range=(0,L_0+100), 
               background_fill_color='#ffffff')
    p.xaxis.axis_label = 'x (m)'
    p.yaxis.axis_label = 'y (m)'
    

    
    # Première trajectoire
    r_1 = p.line('x', 'y', source=source_HAUT,color="#1118cc", line_width=4.5, alpha=0.8, legend='Première boule de neige')

    # Seconde trajectoire 
    r_2 = p.line('x', 'y', source=source_BAS, color="#8778cc", line_width=4.5, alpha=0.8, legend='Deuxième boule de neige')

    point = p.circle('x', 'y', source=source_P, color='#FF3333', legend='Point recherchée')

    # Flèche indiquant la vitesse initiale
    arrow = Arrow(end=OpenHead(line_color="firebrick", ),
                       x_start=0, y_start=0, x_end=10, y_end=10,)
    p.add_layout(arrow)
    
    
    # Définir sliders 
    slider_V = Slider(start=0, end=100, step=0.1,value=20, title='Vitesse initiale')
    slider_L = Slider(start=0, end=100,step=0.1,value=30, title='Distance recherchée')
    
    
    
    def refresh_source(attrname, old, new):
        # Mettre à jour les valeurs à afficher 
        v_0 = slider_V.value
        L_0 = slider_L.value

        HAUT, BAS, P = initialize(L_0, v_0)

        source_HAUT.data = HAUT
        source_BAS.data = BAS
        
        source_P.data = P
        
    slider_V.on_change('value', refresh_source)
    slider_L.on_change('value', refresh_source)
    
    layout = column(
        row(p),
        row(slider_V, slider_L)
    )

    
    doc.add_root(layout)

In [ ]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    
    base_url = os.environ['EXTERNAL_URL']
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url



def show_document(doc):
    servers = list(notebookapp.list_running_servers())[0]
    if servers['hostname'] == 'localhost':
        show(doc) 
    else:
        show(doc, notebook_url=remote_jupyter_proxy_url)

In [ ]:
show_document(modify_doc) 

In [ ]:
# direction confusing - arrow
# not the same time of launching
# choose delay and stop the curve of the second at the moment that the first arrives
# first slider to play with and then generate a video to show both balls

## Trajectoire en fonction de l'angle et le la vitesse initiale

In [ ]:
# On répète 
L_ini = 30 
v_0_ini = 20
alpha_1 = 30 * np.pi/180
alpha_2 = 60 * np.pi/180
    
alpha_high = np.max([alpha_1, alpha_2])
alpha_low = np.min([alpha_1, alpha_2])

L = np.sin(2*alpha_1)*v_0_ini**2/g
    
tf1 = L/(np.cos(alpha_high)*v_0_ini)
tf2 = L/(np.cos(alpha_low)*v_0_ini)

t1 = np.linspace(0,tf1,1000)
t2 = np.linspace(0,tf2,1000)

x_1 = v_0_ini*np.cos(alpha_high)*t1
y_1 = -0.5 *g*t1**2 + v_0_ini*np.sin(alpha_high)*t1

x_2 = v_0_ini*np.cos(alpha_low)*t2
y_2 = -0.5 *g*t2**2 + v_0_ini*np.sin(alpha_low)*t2

f = figure(title="Trajectoire du projectile", plot_height=360, plot_width=720, y_range=(0,np.max([y_1])+20), x_range=(0,L_ini+100), 
           background_fill_color='#ffffff')

# Première trajectoire
r_1 = f.line(x_1, y_1, color="#1118cc", line_width=4.5, alpha=0.8, legend='Première boule de neige')

# Seconde trajectoire 
r_2 = f.line(x_2, y_2, color="#8778cc", line_width=4.5, alpha=0.8, legend='Deuxième boule de neige')

o = f.circle([L_ini], [0], size=20, color="red", alpha=0.5)

f.xaxis.axis_label = 'x (m)'
f.yaxis.axis_label = 'y (m)'

arro = Arrow(end=OpenHead(line_color="firebrick", ),
                   x_start=0, y_start=0, x_end=10, y_end=10)
f.add_layout(arro)

L = np.sin(2*alpha_1)*v_0_ini**2/g
tf1 = L/(np.cos(alpha_1)*v_0_ini)
tf2 = L/(np.cos(alpha_2)*v_0_ini)
    
# tf1_label = Label(x=0, y=220, x_units='screen', y_units='screen', render_mode = 'css',
#                 text='Time of flight first trajectory: {:02.2f} s'.format(tf1),border_line_color='black', border_line_alpha=1.0)
# tf2_label = Label(x=0, y=190, x_units='screen', y_units='screen', render_mode = 'css',
#                text='Time of flight second trajectory: {:02.2f} s'.format(tf2),border_line_color='black', border_line_alpha=1.0)

#f.add_layout(tf1_label)
#f.add_layout(tf2_label)


def update_length(v_0, angle, objectif_distance):
    alpha_1 = (45 + angle) * np.pi/180
    alpha_2 = (45 - angle) * np.pi/180
    
    alpha_high = np.max([alpha_1, alpha_2])
    alpha_low = np.min([alpha_1, alpha_2])
    
    L = np.sin(2*alpha_1)*v_0**2/g
    
    tf1 = L/(np.cos(alpha_1)*v_0)
    tf2 = L/(np.cos(alpha_2)*v_0)
    
    t1 = np.linspace(0,tf1,1000)
    t2 = np.linspace(0,tf2,1000)

   
    r_1.data_source.data['x'] = v_0*np.cos(alpha_high)*t1
    r_1.data_source.data['y'] = -0.5 *g*t1**2 + v_0*np.sin(alpha_high)*t1

    r_2.data_source.data['x'] = v_0*np.cos(alpha_low)*t2
    r_2.data_source.data['y'] = -0.5 *g*t2**2 + v_0*np.sin(alpha_low)*t2
    
        
    o.data_source.data['x'] = [objectif_distance]
    
    arro.x_end = v_0*np.cos(alpha_ini)
    arro.y_end = v_0*np.sin(alpha_ini)
    arro.x_start = 0
    arro.y_start = 0
    
   
    #tf1_label.text = 'Time of flight first trajectory: {:02.2f} s'.format(tf1)
    #tf2_label.text = 'Time of flight second trajectory: {:02.2f} s'.format(tf2)
    print('Time of flight first trajectory: {:02.2f} s'.format(tf1))
    print('Time of flight second trajectory: {:02.2f} s'.format(tf2))

    push_notebook()  
    
show(f, notebook_handle=True)
interact(update_length, \
         v_0=widgets.FloatSlider(min=0,max=100,step=0.1,value=20, description='$v_0$:'), \
        angle=widgets.FloatSlider(min=0,max=45,step=0.01,value=5, description='Angle:'),\
        objectif_distance = widgets.FloatSlider(min=0,max=100,step=0.1,value=5, description='Objectif:') )